In [ ]:
%matplotlib inline

import math
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
import plot

In [ ]:
def fmt_money(number):
    return "${:,.0f}".format(number)

In [ ]:
def run_pmt(market, pmt_rate):
    portfolio = 1_000_000
    age = 65
    max_age = 100
    df = pandas.DataFrame(index=range(age, max_age), columns=['withdrawal', 'portfolio'])
    for i in range(age, max_age):
        withdraw = -numpy.pmt(pmt_rate, max_age-i, portfolio, 0, 1)
        portfolio -= withdraw
        portfolio *= (1 + market)
        df.loc[i] = [int(withdraw), int(portfolio)]
    return df

In [ ]:
pmt_df = run_pmt(0.03, 0.04)
pmt_df.head()

In [ ]:
def run_smile(target):
    spend = target
    s = pandas.Series(index=range(66,100), dtype=int)
    for age in range(66, 100):
        d = (0.00008 * age * age) - (0.0125 * age) - (0.0066 * math.log(target)) + 0.546
        spend *= (1 + d)
        s.loc[age] = int(spend)
    return s

In [ ]:
smile_s = run_smile(pmt_df.iloc[0]['withdrawal'])
smile_s.head()

In [ ]:
def rmse(s1, s2):
    return numpy.sqrt(numpy.mean((s1-s2)**2))

In [ ]:
rmse(pmt_df['withdrawal'][1:26], smile_s[:26])

In [ ]:
def harness():
    df = pandas.DataFrame(columns=['market', 'pmtrate', 'rmse'])
    for returns in numpy.arange(0.01, 0.10+0.001, 0.001):
        for pmt_rate in numpy.arange(0.01, 0.10+0.001, 0.001):
            pmt_df = run_pmt(returns, pmt_rate)
            iwd = pmt_df.iloc[0]['withdrawal']
            smile_s = run_smile(iwd)
            errors = rmse(pmt_df['withdrawal'], smile_s)
            df = df.append({'market': returns, 'pmtrate': pmt_rate, 'rmse': errors}, ignore_index=True)
    return df

In [ ]:
error_df = harness()
error_df.head()

In [ ]:
#seaborn.scatterplot(data=error_df, x='market', y='pmtrate', size='rmse')

In [ ]:
#seaborn.scatterplot(data=error_df[0:19], x='pmtrate', y='rmse')

In [ ]:
error_df[0:91]

In [ ]:
slice_size = 91
n_slices = int(len(error_df) / slice_size)
print(len(error_df), n_slices, slice_size)
for i in range(n_slices):
    start = i * slice_size
    end = i * slice_size + slice_size
    slice_df = error_df[start:end]
    delta = slice_df['pmtrate'] - slice_df['market']
    plot_df = pandas.DataFrame({'delta': delta, 'rmse': slice_df['rmse']})
    sp = seaborn.scatterplot(data=plot_df, x='delta', y='rmse')
    mkt_rate = slice_df.iloc[0]['market']
    plt.xticks(numpy.arange(-0.100, +0.100, 0.005), rotation='vertical')
#    plt.title(f'Market returns: {mkt_rate*100}%')

In [ ]:
series = pandas.Series(index=range(40_000, 101_000, 5_000))
for t in range(40_000, 101_000, 5_000):
    s = run_smile(t)
    contingency = (t - s[0:20]).sum()
    series.loc[t] = contingency

In [ ]:
series.plot()
plt.xlabel('Targeted annual withdrawal at retirement')
plt.ylabel('Contigency fund')
xticks = plt.xticks()
plt.xticks(xticks[0], [fmt_money(x) for x in xticks[0]])
yticks = plt.yticks()
plt.yticks(yticks[0], [fmt_money(y) for y in yticks[0]])
plt.title('Contigency at age 85')

In [ ]:
series

In [ ]:
(series / series.index).plot()
plt.title('Ratio of contingency to expected spending')
xticks = plt.xticks()
plt.xticks(xticks[0], [fmt_money(x) for x in xticks[0]])

In [ ]:
len(error_df)